## EJERCICIO 4

Desde la empresa nos piden que realicemos unas tareas de investigación y práctica sobre PostgreSQL, como nuevos expertos que sois.


En concreto se nos pide explicar a qué nos referimos cuando decimos que una función en PostgreSQL es volátil por defecto, y enumerar las posibles alternativas que tenemos.


La explicación debe acompañarse con ejemplos simples que ayuden a facilitar la comprensión de la respuesta.

> Cada función tiene una clasificación de volatilidad, siendo las posibilidades VOLÁTIL, ESTABLE o INMUTABLE.   
>  
> __VOLATILE__ es el valor predeterminado si el comando __CREATE FUNCTION__ no especifica una categoría.   
> La categoría de volatilidad es una promesa para el optimizador sobre el comportamiento de la función:
>
> * Una __función VOLATIL__ puede hacer cualquier cosa, incluida la modificación de la base de datos. Puede devolver diferentes resultados en sucesivas llamadas con los mismos argumentos. El optimizador no hace suposiciones sobre el comportamiento de tales funciones. Una consulta que utilice una función volátil volverá a evaluar la función en cada fila donde se necesite su valor.
>  
>  
> * Una __función STABLE__ no puede modificar la base de datos y se garantiza que devolverá los mismos resultados dados los mismos argumentos para todas las filas dentro de una sola declaración. Esta categoría permite al optimizador optimizar múltiples llamadas de la función a una sola llamada. En particular, es seguro utilizar una expresión que contenga dicha función en una condición de escaneo de índice. (Dado que un escaneo de índice evaluará el valor de comparación solo una vez, no una vez en cada fila, no es válido usar una función VOLATILE en una condición de escaneo de índice).
>
>  
> * Una __función INMUTABLE__ no puede modificar la base de datos y se garantiza que devolverá los mismos resultados dados los mismos argumentos para siempre. Esta categoría permite al optimizador evaluar previamente la función cuando una consulta la llama con argumentos constantes. Por ejemplo, una consulta como SELECT ... WHERE x = 2 + 2 se puede simplificar a la vista para SELECT ... WHERE x = 4, porque la función subyacente al operador de suma de enteros está marcada como INMUTABLE.
>
>
> Para obtener los mejores resultados de optimización, debe etiquetar sus funciones con la categoría de volatilidad más estricta que sea válida para ellas.
>
>
> Cualquier función con efectos secundarios debe etiquetarse como VOLÁTIL, de modo que las llamadas a ella no se puedan optimizar. Incluso una función sin efectos secundarios debe etiquetarse como VOLÁTIL si su valor puede cambiar dentro de una sola consulta; algunos ejemplos son random (), currval (), timeofday ().   
>  
> Otro ejemplo importante es que la familia de funciones current_timestamp califica como STABLE, ya que sus valores no cambian dentro de una transacción.
>  
>  
> Hay relativamente poca diferencia entre las categorías ESTABLE e INMUTABLE cuando se consideran consultas interactivas simples que se planifican y ejecutan de inmediato: no importa mucho si una función se ejecuta una vez durante la planificación o una vez durante el inicio de la ejecución de la consulta. Pero hay una gran diferencia si el plan se guarda y se reutiliza más tarde. Etiquetar una función como INMUTABLE cuando en realidad no lo es podría permitir que se pliegue prematuramente a una constante durante la planificación, lo que da como resultado que un valor obsoleto se reutilice durante los usos posteriores del plan. Esto es un peligro cuando se usan declaraciones preparadas o cuando se usan lenguajes de funciones que almacenan planes en caché (como PL / pgSQL). 
>  
>  
> Para las funciones escritas en SQL o en cualquiera de los lenguajes de procedimiento estándar, existe una segunda propiedad importante determinada por la categoría de volatilidad, a saber, la visibilidad de cualquier cambio de datos que haya realizado el comando SQL que llama a la función. Una función VOLÁTIL verá tales cambios, una función ESTABLE o INMUTABLE no lo hará. Este comportamiento se implementa usando el comportamiento de instantánea de MVCC (ver Capítulo 13): las funciones STABLE e INMUTABLE usan una instantánea establecida al inicio de la consulta que llama, mientras que las funciones VOLATILE obtienen una instantánea nueva al comienzo de cada consulta que ejecutan.
>
>
> Debido a este comportamiento de creación de instantáneas, una función que contenga solo comandos SELECT puede marcarse con seguridad como ESTABLE, incluso si selecciona de tablas que podrían estar sufriendo modificaciones por consultas simultáneas. PostgreSQL ejecutará todos los comandos de una función STABLE utilizando la instantánea establecida para la consulta que realiza la llamada, por lo que verá una vista fija de la base de datos a lo largo de esa consulta.  
>   
> El mismo comportamiento de captura de instantáneas se utiliza para los comandos SELECT dentro de las funciones INMUTABLE. Por lo general, no es aconsejable seleccionar de las tablas de la base de datos dentro de una función INMUTABLE, ya que la inmutabilidad se romperá si el contenido de la tabla alguna vez cambia. Sin embargo, PostgreSQL no exige que usted no haga eso.   
>  
> Un error común es etiquetar una función como INMUTABLE cuando sus resultados dependen de un parámetro de configuración. Por ejemplo, una función que manipula marcas de tiempo bien podría tener resultados que dependan de la configuración de la zona horaria. Por razones de seguridad, estas funciones deben etiquetarse como ESTABLE en su lugar.